# Read LD matrix

In [1]:
import numpy as np

In [2]:
files = [
    '../phase2/ld/eas.eur.22.0.ld.bin',
    '../phase2/ld/eas.eur.22.1.ld.bin' ]

In [3]:
Sigmas = [ np.fromfile(f, 'double', -1, '') for f in files ]

In [4]:
Ms = [ int(np.sqrt(Sigma.shape[0])) for Sigma in Sigmas ]
M = 0

if (Ms[0] == Ms[1]):
    M = Ms[0]
    print("M={}".format(M))
else:
    print("Ms not equal")

M=2877


In [5]:
Sigmas = np.array([ Sigma.reshape((M,M)) for Sigma in Sigmas ])

# Massage LD matrix

In [6]:
I = np.diag(np.ones(M))

In [7]:
l = 0.0
Sigma_primes = Sigmas + np.array([l*I,l*I])

# Generate genotypes

In [8]:
import numpy.random as nr

In [9]:
N = 100

In [10]:
Xs = np.array([nr.multivariate_normal(np.zeros(M), Sigma, N) for Sigma in Sigma_primes])

C:\Users\Kevin\Miniconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: covariance is not positive-semidefinite.
  if __name__ == '__main__':


# Generate effect sizes

In [11]:
Mc = 30

In [12]:
rho_b = 0.8

In [13]:
def corr_matrix(rho_b, M=2):
    a = (1-rho_b)*np.diag(np.ones(M))
    b = rho_b*np.ones_like(a)
    return(a+b)

In [14]:
bs = nr.multivariate_normal(np.zeros(2), corr_matrix(rho_b), Mc)

In [15]:
Is = nr.choice(M, Mc, False)
Is.sort()

In [16]:
mask_causal = np.zeros(M, dtype='bool')
mask_causal[Is] = True
mask_tagged = ~ mask_causal

# Generate phenotypes

In [17]:
Ys = np.array([X[:,mask_causal].dot(b) for X, b in zip(Xs, bs.T)])

# True g

$
\begin{align}
g_i & = \Sigma_{TT}^{-1} \Sigma_{TC} b
\end{align}
$

In [28]:
gs = np.array(
    [
        np.linalg.inv(
            Sigma[mask_tagged][:,mask_tagged]
        ).dot(
            Sigma[mask_tagged][:,mask_causal].dot(b)
            ) for b, Sigma in zip(bs.T, Sigma_primes)])

In [31]:
np.corrcoef(bs.T)

array([[ 1.        ,  0.67956291],
       [ 0.67956291,  1.        ]])

In [32]:
np.corrcoef(gs)

array([[ 1.       ,  0.1792467],
       [ 0.1792467,  1.       ]])